In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping

import cv2
import os
from imutils import paths
import imutils
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
import models
from models import SimplePreprocessor, AnimalsDatasetManager



In [14]:
def build_cnn():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3,3), padding='same', activation= 'relu', input_shape=(32,32,3)))
    model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(layers.AveragePooling2D(pool_size=(2, 2), padding='same'))
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.AveragePooling2D(pool_size=(2, 2), padding='same'))
    model.add(layers.Flatten())
    model.add(layers.Dense(10, activation='softmax'))
    return model

In [15]:
def create_label_folder_dict(adir):
    sub_folders= [folder for folder in os.listdir(adir)
                  if os.path.isdir(os.path.join(adir, folder))]
    label_folder_dict= dict()
    for folder in sub_folders:
        item= {folder: os.path.abspath(os.path.join(adir, folder))}
        label_folder_dict.update(item)
    return label_folder_dict

In [16]:
label_folder_dict= create_label_folder_dict("../../datasets/Animals/")

In [17]:
sp = SimplePreprocessor(width=32, height=32)
data_manager = AnimalsDatasetManager([sp])
data_manager.load(label_folder_dict, verbose=100)
data_manager.process_data_label()
data_manager.train_valid_test_split()

butterfiles 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
cats 501
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
chickens 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
cows 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
dogs 501
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
elephants 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
fishes 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
horses 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
lions 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
spiders 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Proc

In [ ]:
batch_size = 32
img_height = 32
img_width = 32

# def configure_for_performance(ds):
#     ds = ds.shuffle(buffer_size=1000)   # shuffle the dataset
#     ds = ds.batch(batch_size)    # partition the dataset to obtain a batch dataset
#     ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)  # prefetch some batches for faster processing
#     return ds

# train_ds = configure_for_performance(train_ds)
# val_ds = configure_for_performance(val_ds)

In [18]:


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, models, layers, regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from functools import partial
from models import DefaultModel


# Load pretrained model

In [54]:
from tensorflow.keras.applications import VGG19

input_shape = (32,32,3)

print("Loading network...")
base_model = VGG19(weights="imagenet", include_top=False, input_tensor=tf.keras.layers.Input(shape=input_shape))
print("Showing layers...")
# loop over the layers in the network and display them to the console
for (i, layer) in enumerate(base_model.layers):
    print("{}\t{}".format(i, layer.__class__.__name__))

Loading network...
Showing layers...
0	InputLayer
1	Conv2D
2	Conv2D
3	MaxPooling2D
4	Conv2D
5	Conv2D
6	MaxPooling2D
7	Conv2D
8	Conv2D
9	Conv2D
10	Conv2D
11	MaxPooling2D
12	Conv2D
13	Conv2D
14	Conv2D
15	Conv2D
16	MaxPooling2D
17	Conv2D
18	Conv2D
19	Conv2D
20	Conv2D
21	MaxPooling2D


In [55]:

class FCHeadNet:
    @staticmethod
    def build(base_model, n_classes=5, D=100): # initialize the head model that will be placed on top of the base, then add a FC layer
        
        base_model.trainable= False # only allow new classifer on top of head to be update
        head_model = base_model.output
        head_model = tf.keras.layers.Flatten(name="flatten")(head_model)
        head_model = tf.keras.layers.Dense(D, activation="relu")(head_model)
        head_model = tf.keras.layers.Dropout(0.3)(head_model)
        # add a softmax layer
        head_model = tf.keras.layers.Dense(n_classes, activation="softmax")(head_model)
        return head_model

In [56]:
head_model = FCHeadNet.build(base_model, len(data_manager.classes), 256)
model = tf.keras.models.Model(inputs=base_model.input, outputs=head_model)



In [57]:
opt = tf.keras.optimizers.RMSprop(lr=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

c:\Users\PC\anaconda3\envs\tf2_cpu\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


# Training model

In [58]:
# model = build_cnn()

### We train the new model on our dataset. 
##### Note that at the moment, we only update the classifier on the top. Owning the existed patterns of `VGG19`, we can obtain a good performance.

In [59]:
# opt = tf.keras.optimizers.Adam(learning_rate= 0.001)
# model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# early_checkpoint = EarlyStopping(patience=5, monitor='val_accuracy', mode='max', restore_best_weights=True) # same as monitoring min for val_loss
# callbacks = [early_checkpoint]

# history = model.fit(x = data_manager.X_train, y = data_manager.y_train, 
#                     validation_data = (data_manager.X_valid, data_manager.y_valid), 
#                     callbacks=callbacks,
#                     epochs = 30, batch_size = 64)

model.fit(  x = data_manager.X_train, y = data_manager.y_train,
            validation_data = (data_manager.X_valid, data_manager.y_valid), 
            epochs=5, batch_size = 64)



Epoch 1/5
63/63 [==============================] - 82s 1s/step - loss: 1.7274 - accuracy: 0.4008 - val_loss: 1.3752 - val_accuracy: 0.5420
Epoch 2/5
63/63 [==============================] - 76s 1s/step - loss: 1.4012 - accuracy: 0.5210 - val_loss: 1.2857 - val_accuracy: 0.5720
Epoch 3/5
63/63 [==============================] - 74s 1s/step - loss: 1.2859 - accuracy: 0.5550 - val_loss: 1.2932 - val_accuracy: 0.5520
Epoch 4/5
63/63 [==============================] - 67s 1s/step - loss: 1.2044 - accuracy: 0.5875 - val_loss: 1.2409 - val_accuracy: 0.5680
Epoch 5/5
63/63 [==============================] - 72s 1s/step - loss: 1.1383 - accuracy: 0.6143 - val_loss: 1.2140 - val_accuracy: 0.6000


### We now train both the feature extractor and classifier with a small learning rate.

In [60]:
base_model.trainable = True
# model.fit(train_ds, epochs=2, validation_data=val_ds)
model.fit(  x = data_manager.X_train, y = data_manager.y_train, 
            validation_data = (data_manager.X_valid, data_manager.y_valid), 
            epochs=2, 
            batch_size = 64)

Epoch 1/2
63/63 [==============================] - 75s 1s/step - loss: 1.0893 - accuracy: 0.6215 - val_loss: 1.2571 - val_accuracy: 0.5780
Epoch 2/2
63/63 [==============================] - 72s 1s/step - loss: 1.0319 - accuracy: 0.6400 - val_loss: 1.2217 - val_accuracy: 0.5720


In [61]:
model.save("./models/first_model", save_format="tf")

INFO:tensorflow:Assets written to: ./models/first_model\assets


# Load Test Set

In [62]:
categorical_labels = ['butterfly', 'cat', 'chicken', 'cow', 'dog', 'elephant', 'fish', 'horse', 'lion', 'spider']

In [63]:
def load_test_set(folder):
    from PIL import Image
    from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
    image_paths = sorted(list(paths.list_images(folder)))
    test_data = []
    for i, image_path in enumerate(image_paths):
        img = Image.open(image_path).convert("RGB") #load_img(image_path)
        img = img.resize((32, 32), Image.ANTIALIAS)
        x = img_to_array(img)
        test_data.append(x)
        if i+1 % 500 ==0:
            print("Loaded {} images".format(i+1))
    test_data = np.asarray(test_data)
    return test_data

In [64]:
test_data = load_test_set("./official_test_aug/")

C:\Users\PC\AppData\Local\Temp\ipykernel_20020\1344258102.py:8: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((32, 32), Image.ANTIALIAS)


In [65]:
print("Test shape: {}".format(test_data.shape))

Test shape: (23098, 32, 32, 3)


In [66]:
preds = model.predict(test_data, batch_size= 32)

print(preds.shape)
print(preds[0:15])

(23098, 10)
[[9.99680519e-01 0.00000000e+00 3.19480052e-04 4.03623806e-17
  0.00000000e+00 0.00000000e+00 0.00000000e+00 5.94290569e-35
  0.00000000e+00 0.00000000e+00]
 [2.24729140e-17 0.00000000e+00 0.00000000e+00 0.00000000e+00
  6.65556180e-18 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.00000000e+00 2.01381660e-14]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.91773756e-03 2.16239141e-06 1.83330510e-32 0.00000000e+00
  9.95080113e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.36580491e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  2.08971198e-22 0.00000000e+00 2.63419479e-01 0.00000000e+00
  0.00000000e+00 4.47717989e-36]
 [0.00000000e+00 0.00000000e+00 1.10547660e-12 1.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 3.16966245e-12
  6.34815415e-14 0.00000000e+00]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00

In [67]:
num_predicted_labels = np.argmax(preds, axis=1)
# num_predicted_labels = yourmodel.predict(test_data)

print(num_predicted_labels[0:15])

for elem in num_predicted_labels:
    print(elem)


[0 8 0 4 0 3 0 1 2 3 7 7 2 2 2]
0
8
0
4
0
3
0
1
2
3
7
7
2
2
2
1
2
2
5
4
3
7
1
2
4
2
2
7
2
2
1
7
0
5
0
3
0
0
2
4
2
1
3
9
2
2
2
2
6
6
4
0
3
4
0
4
1
0
0
1
1
2
4
1
3
3
2
9
5
1
0
5
4
3
0
1
4
1
4
7
1
0
7
4
2
2
0
6
3
0
4
2
4
1
2
3
0
7
5
2
0
3
4
0
7
1
1
0
2
0
7
0
2
1
1
8
2
1
1
2
2
6
6
6
7
1
6
3
1
6
2
4
5
7
0
0
3
0
2
2
3
6
9
0
4
0
2
3
7
6
0
1
4
7
8
1
0
4
2
4
6
4
0
5
0
9
2
7
1
5
6
9
2
0
1
0
1
7
4
0
4
6
0
6
1
7
0
2
0
5
4
6
0
0
2
0
2
7
1
7
6
6
0
0
0
9
7
2
0
1
0
2
9
1
7
3
5
0
1
6
0
2
3
1
1
3
1
0
2
4
7
0
7
0
3
0
0
7
0
0
5
6
1
0
0
4
7
4
0
0
4
0
0
0
0
1
9
3
6
1
1
7
3
3
7
4
2
9
2
9
6
0
0
3
3
3
2
3
0
3
0
2
0
7
5
6
1
2
1
0
3
8
8
2
2
4
7
8
3
5
0
7
4
6
4
0
9
2
0
4
0
2
8
5
4
2
2
6
7
4
6
3
7
3
7
0
2
1
0
7
2
2
0
3
2
6
0
5
1
7
0
0
0
3
0
3
0
2
0
4
2
3
4
2
2
6
1
2
0
0
8
0
6
0
2
8
2
0
2
8
2
6
0
0
0
2
7
2
1
3
0
0
0
7
6
4
6
1
9
7
1
9
6
6
7
7
0
0
1
9
6
5
1
0
9
1
1
2
1
4
8
0
5
0
6
3
7
2
6
3
9
2
0
0
1
9
0
9
2
4
2
6
6
2
2
6
0
0
6
6
4
9
0
0
9
9
4
9
1
5
0
0
8
2
3
5
1
5
7
6
4
7
9
1
4
8
0
4
2
5
0
7
6
6
2
4
2
0
1
0
1
3
1
0


In [68]:
cat_predicted_labels= [categorical_labels[num_predicted_labels[i]] for i in range(len(num_predicted_labels))]
print(cat_predicted_labels[0:15])



['butterfly', 'lion', 'butterfly', 'dog', 'butterfly', 'cow', 'butterfly', 'cat', 'chicken', 'cow', 'horse', 'horse', 'chicken', 'chicken', 'chicken']


In [69]:
def create_solution():
    import csv
    header= ['ID', 'Label']
    with open('my_solution.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        # write the header
        writer.writerow(header)
        data = []
        for i in range(len(cat_predicted_labels)):
            data.append([str(i), cat_predicted_labels[i]])
        writer.writerows(data)

In [70]:
create_solution()

You now can submit the csv file `my_solution.csv` to the Kaggle competition.